# 📖 DATA STORYTELLING AGENT - FIXED VERSION ✅

**Works with YouTube Transcript API v1.2.0+ and Google GenAI SDK**

## 🚀 Quick Start:
1. Run cells in order
2. Add your API key in Step 3
3. Generate posts!

---

## 📦 Step 1: Install Packages

In [1]:
# Install the latest versions
!pip install google-genai youtube-transcript-api --upgrade --quiet

print("✅ Packages installed successfully!")
print("\n📋 Versions:")
!pip show google-genai | grep Version
!pip show youtube-transcript-api | grep Version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.1/485.1 kB 14.4 MB/s eta 0:00:00
✅ Packages installed successfully!

📋 Versions:
Version: 1.47.0
Version: 1.2.3


## 📚 Step 2: Import Libraries and Define Classes

In [2]:
import os
import json
import re
import logging
import traceback
from dataclasses import dataclass, field
from typing import List, Dict, Optional
from datetime import datetime
from enum import Enum

# YouTube Transcript API - FIXED imports (no TooManyRequests)
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import (
    TranscriptsDisabled,
    NoTranscriptFound,
    VideoUnavailable,
    YouTubeRequestFailed
)

# New Google GenAI SDK
from google import genai
from google.genai import types

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## 🏗️ Step 3: Define Error Classes & Data Structures

In [3]:
# ============================================================================
# ERROR CLASSES
# ============================================================================

class DataStorytellingError(Exception):
    """Base exception"""
    pass

class TranscriptExtractionError(DataStorytellingError):
    """Transcript error"""
    pass

class APIError(DataStorytellingError):
    """API error"""
    pass

class PostGenerationError(DataStorytellingError):
    """Post generation error"""
    pass


# ============================================================================
# DATA STRUCTURES
# ============================================================================

@dataclass
class DataSource:
    """Verified data source"""
    claim: str
    source: str
    reliability_score: float
    context: str = ""

@dataclass
class StoryStructure:
    """Narrative structure"""
    hook: str
    context: str
    revelation: str
    consequences: str
    data_sources: List[DataSource] = field(default_factory=list)

@dataclass
class SocialPost:
    """Social media post with metrics"""
    platform: str
    content: str
    story_structure: StoryStructure
    quality_score: float
    data_reliability_score: float
    audience_engagement_score: float
    narrative_flow_score: float

class Platform(Enum):
    """Platforms"""
    INSTAGRAM = "instagram"
    LINKEDIN = "linkedin"

print("✅ Error classes and data structures defined!")

✅ Error classes and data structures defined!


## 🎬 Step 4: Transcript Extractor Class (FIXED)

In [4]:
class TranscriptExtractor:
    """Handles YouTube transcript extraction - FIXED for API v1.2.0+"""

    def __init__(self):
        self.logger = logging.getLogger(__name__ + '.TranscriptExtractor')

    def extract_video_id(self, url_or_id: str) -> str:
        """Extract video ID from URL"""
        try:
            if re.match(r'^[a-zA-Z0-9_-]{11}$', url_or_id):
                return url_or_id

            patterns = [
                r'(?:youtube\.com\/watch\?v=|youtu\.be\/)([a-zA-Z0-9_-]{11})',
                r'youtube\.com\/embed\/([a-zA-Z0-9_-]{11})',
            ]

            for pattern in patterns:
                match = re.search(pattern, url_or_id)
                if match:
                    return match.group(1)

            raise TranscriptExtractionError(f"Invalid video ID/URL: {url_or_id}")
        except Exception as e:
            raise TranscriptExtractionError(f"Failed to extract video ID: {str(e)}")

    def get_transcript(self, video_id: str, languages: List[str] = None) -> str:
        """
        Get transcript - FIXED for new API structure

        IMPORTANT: The new API (v1.2.0+) returns FetchedTranscript with .snippets
        Each snippet has .text, .start, .duration attributes (NOT dictionary)
        """
        if languages is None:
            languages = ['en']

        try:
            self.logger.info(f"Fetching transcript for: {video_id}")

            # Create API instance
            api = YouTubeTranscriptApi()

            # Fetch transcript - returns FetchedTranscript object
            try:
                fetched_transcript = api.fetch(video_id, languages=languages)
                self.logger.info(f"Got transcript in: {fetched_transcript.language}")
            except NoTranscriptFound:
                self.logger.warning(f"No transcript in {languages}, trying any language")
                fetched_transcript = api.fetch(video_id)
                self.logger.info(f"Using transcript in: {fetched_transcript.language}")

            # FIXED: Access snippets correctly
            # OLD WAY (doesn't work): entry['text']
            # NEW WAY (works): snippet.text
            full_text = " ".join([snippet.text for snippet in fetched_transcript.snippets])

            self.logger.info(f"✅ Transcript extracted: {len(full_text)} characters")
            self.logger.info(f"✅ Number of snippets: {len(fetched_transcript.snippets)}")

            return full_text

        except TranscriptsDisabled:
            raise TranscriptExtractionError(f"Transcripts disabled for video {video_id}")
        except VideoUnavailable:
            raise TranscriptExtractionError(f"Video {video_id} is unavailable")
        except YouTubeRequestFailed as e:
            raise TranscriptExtractionError(f"YouTube request failed: {str(e)}")
        except Exception as e:
            error_msg = f"Unexpected error: {str(e)}\n{traceback.format_exc()}"
            self.logger.error(error_msg)
            raise TranscriptExtractionError(error_msg)

print("✅ TranscriptExtractor class defined!")

✅ TranscriptExtractor class defined!


## 🤖 Step 5: Gemini API Client

In [5]:
class GeminiClient:
    """Gemini API client using NEW SDK"""

    def __init__(self, api_key: str, model_name: str = "gemini-2.0-flash"):
        self.logger = logging.getLogger(__name__ + '.GeminiClient')
        self.api_key = api_key
        self.model_name = model_name

        try:
            self.client = genai.Client(api_key=api_key)
            self.logger.info(f"✅ Gemini client initialized: {model_name}")

            # Test API key
            test_response = self.client.models.generate_content(
                model=model_name,
                contents="Hello",
                config=types.GenerateContentConfig(max_output_tokens=10)
            )
            self.logger.info("✅ API key validated")

        except Exception as e:
            raise APIError(f"Failed to initialize: {str(e)}")

    def generate_content(self, prompt: str, temperature: float = 0.7) -> str:
        """Generate content"""
        try:
            self.logger.info("Generating content...")

            response = self.client.models.generate_content(
                model=self.model_name,
                contents=prompt,
                config=types.GenerateContentConfig(
                    temperature=temperature,
                    max_output_tokens=2048,
                )
            )

            if hasattr(response, 'text'):
                result = response.text
            elif hasattr(response, 'candidates') and response.candidates:
                result = response.candidates[0].content.parts[0].text
            else:
                raise APIError("Unexpected response format")

            self.logger.info(f"✅ Generated: {len(result)} characters")
            return result

        except Exception as e:
            raise APIError(f"Generation failed: {str(e)}")

print("✅ GeminiClient class defined!")

✅ GeminiClient class defined!


## 📖 Step 6: Data Storytelling Engine

In [6]:
class DataStorytellingEngine:
    """Creates data-driven stories"""

    def __init__(self, gemini_client: GeminiClient):
        self.client = gemini_client
        self.logger = logging.getLogger(__name__ + '.Engine')

    def analyze_transcript(self, transcript: str) -> StoryStructure:
        """Analyze transcript for story structure"""
        try:
            self.logger.info("Analyzing transcript...")

            prompt = f"""Analyze this transcript and extract a data story in JSON format.

TRANSCRIPT:
{transcript[:5000]}

Return JSON only:
{{
    "hook": "Attention-grabbing statement (1-2 sentences)",
    "context": "Background (2-3 sentences)",
    "revelation": "Key insight (1-2 sentences)",
    "consequences": "Why it matters (2-3 sentences)",
    "key_data_points": [
        {{"claim": "specific claim", "source": "transcript", "context": "context"}}
    ]
}}

Requirements:
- Focus on concrete data and facts
- Make relevant to professionals 20-35
- Ensure claims backed by transcript"""

            response = self.client.generate_content(prompt, temperature=0.3)

            # Parse JSON
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if not json_match:
                raise PostGenerationError("No JSON in response")

            data = json.loads(json_match.group(0))

            # Create structure
            data_sources = []
            for dp in data.get('key_data_points', []):
                data_sources.append(DataSource(
                    claim=dp.get('claim', ''),
                    source='transcript',
                    reliability_score=0.9,
                    context=dp.get('context', '')
                ))

            structure = StoryStructure(
                hook=data.get('hook', ''),
                context=data.get('context', ''),
                revelation=data.get('revelation', ''),
                consequences=data.get('consequences', ''),
                data_sources=data_sources
            )

            self.logger.info(f"✅ Analyzed: {len(data_sources)} data points")
            return structure

        except Exception as e:
            raise PostGenerationError(f"Analysis failed: {str(e)}")

    def generate_post(self, story: StoryStructure, platform: Platform) -> SocialPost:
        """Generate social media post"""
        try:
            self.logger.info(f"Generating {platform.value} post...")

            if platform == Platform.INSTAGRAM:
                guidelines = """Platform: Instagram
Tone: Casual, engaging
Length: 150-200 words
Emojis: 2-3
Format: Short paragraphs"""
            else:
                guidelines = """Platform: LinkedIn
Tone: Professional
Length: 200-300 words
Emojis: None
Format: Professional paragraphs"""

            data_text = "\n".join([f"- {ds.claim}" for ds in story.data_sources[:3]])

            prompt = f"""Create a compelling social media post.

{guidelines}

STORY:
Hook: {story.hook}
Context: {story.context}
Revelation: {story.revelation}
Consequences: {story.consequences}

DATA:
{data_text}

Requirements:
1. Start with hook
2. Build context
3. Deliver revelation
4. End with consequences
5. Back all claims with data
6. Storytelling format
7. Target: professionals 20-35

Write the post:"""

            content = self.client.generate_content(prompt, temperature=0.7)
            scores = self._calculate_scores(content, story, platform)

            post = SocialPost(
                platform=platform.value,
                content=content,
                story_structure=story,
                quality_score=scores['overall'],
                data_reliability_score=scores['data'],
                audience_engagement_score=scores['engagement'],
                narrative_flow_score=scores['flow']
            )

            self.logger.info(f"✅ Generated: Quality {post.quality_score:.1f}%")
            return post

        except Exception as e:
            raise PostGenerationError(f"Post generation failed: {str(e)}")

    def _calculate_scores(self, content: str, story: StoryStructure, platform: Platform) -> Dict[str, float]:
        """Calculate quality scores"""
        data_score = min(100, len(story.data_sources) * 30)
        word_count = len(content.split())
        target = (150, 200) if platform == Platform.INSTAGRAM else (200, 300)
        engagement = 90 if target[0] <= word_count <= target[1] else 70

        flow = 0
        if story.hook.lower()[:20] in content.lower():
            flow += 25
        if any(w in content.lower() for w in ['because', 'so', "that's why"]):
            flow += 25
        if story.revelation.lower()[:20] in content.lower():
            flow += 25
        if any(w in content.lower() for w in ['matter', 'important', 'means']):
            flow += 25

        overall = (data_score + engagement + flow) / 3

        return {
            'overall': overall,
            'data': data_score,
            'engagement': engagement,
            'flow': flow
        }

print("✅ DataStorytellingEngine class defined!")

✅ DataStorytellingEngine class defined!


## 🎯 Step 7: Main Agent Class

In [7]:
class DataStorytellingAgent:
    """Main agent orchestrating everything"""

    def __init__(self, api_key: str, model_name: str = "gemini-2.0-flash"):
        self.logger = logging.getLogger(__name__ + '.Agent')
        self.logger.info("="*70)
        self.logger.info("INITIALIZING DATA STORYTELLING AGENT")
        self.logger.info("="*70)

        try:
            self.transcript_extractor = TranscriptExtractor()
            self.gemini_client = GeminiClient(api_key, model_name)
            self.engine = DataStorytellingEngine(self.gemini_client)
            self.logger.info("✅ All components initialized")
        except Exception as e:
            self.logger.error(f"Initialization failed: {str(e)}")
            raise

    def create_posts_from_video(
        self,
        video_url_or_id: str,
        platforms: List[Platform] = None,
        languages: List[str] = None
    ) -> Dict[str, SocialPost]:
        """Create posts from video"""
        if platforms is None:
            platforms = [Platform.INSTAGRAM, Platform.LINKEDIN]
        if languages is None:
            languages = ['en']

        self.logger.info("\n" + "="*70)
        self.logger.info("STARTING POST GENERATION")
        self.logger.info("="*70)

        try:
            # Step 1: Extract ID
            self.logger.info("\nSTEP 1: Extracting video ID...")
            video_id = self.transcript_extractor.extract_video_id(video_url_or_id)
            self.logger.info(f"✅ Video ID: {video_id}")

            # Step 2: Get transcript
            self.logger.info("\nSTEP 2: Fetching transcript...")
            transcript = self.transcript_extractor.get_transcript(video_id, languages)
            self.logger.info(f"✅ Transcript: {len(transcript)} characters")

            # Step 3: Analyze
            self.logger.info("\nSTEP 3: Analyzing transcript...")
            story = self.engine.analyze_transcript(transcript)
            self.logger.info(f"✅ Found {len(story.data_sources)} data sources")

            # Step 4: Generate posts
            self.logger.info("\nSTEP 4: Generating posts...")
            posts = {}
            for platform in platforms:
                post = self.engine.generate_post(story, platform)
                posts[platform.value] = post
                self.logger.info(f"  ✅ {platform.value}: {post.quality_score:.1f}%")

            self.logger.info("\n" + "="*70)
            self.logger.info("✅ ALL POSTS GENERATED SUCCESSFULLY!")
            self.logger.info("="*70)

            return posts

        except Exception as e:
            self.logger.error(f"\n❌ ERROR: {str(e)}")
            raise

print("✅ DataStorytellingAgent class defined!")
print("\n" + "="*70)
print("🎉 ALL CLASSES LOADED - READY TO USE!")
print("="*70)

✅ DataStorytellingAgent class defined!

🎉 ALL CLASSES LOADED - READY TO USE!


## 🔑 Step 8: Configure Your API Key

⚠️ **IMPORTANT:** Replace `YOUR_API_KEY_HERE` with your actual Gemini API key!

Get your key here: https://aistudio.google.com/app/apikey

In [8]:
# ⚠️ REPLACE THIS WITH YOUR ACTUAL API KEY!
GEMINI_API_KEY = " "

# Model selection
MODEL_NAME = "gemini-2.0-flash"  # Fast and efficient
# Alternative: "gemini-2.5-pro" for highest quality

# Validate API key format
if GEMINI_API_KEY == "YOUR_API_KEY_HERE":
    print("❌ ERROR: Please replace YOUR_API_KEY_HERE with your actual API key!")
    print("\n📍 Get your key here: https://aistudio.google.com/app/apikey")
elif not GEMINI_API_KEY.startswith("AIzaSy"):
    print("⚠️ WARNING: Your API key doesn't look correct.")
    print("   Gemini API keys typically start with 'AIzaSy'")
else:
    print("✅ API key configured!")
    print(f"✅ Using model: {MODEL_NAME}")

✅ API key configured!
✅ Using model: gemini-2.0-flash


## 🧪 Step 9: Test with a Video

Run this cell to generate your first posts! 🎉

In [9]:
# Verified working video IDs
VIDEO_ID = "Ks-_Mh1QhMc"  # Simon Sinek - How great leaders inspire action
# Other options:
# VIDEO_ID = "Ks-_Mh1QhMc"  # Amy Cuddy - Body Language
# VIDEO_ID = "arj7oStGLkU"  # Tim Urban - Procrastination

try:
    print("\n🚀 Initializing agent...\n")
    agent = DataStorytellingAgent(api_key=GEMINI_API_KEY, model_name=MODEL_NAME)

    print("\n📝 Generating posts...\n")
    posts = agent.create_posts_from_video(
        video_url_or_id=VIDEO_ID,
        platforms=[Platform.INSTAGRAM, Platform.LINKEDIN],
        languages=['en']
    )

    # Display results
    print("\n" + "="*70)
    print("📊 RESULTS")
    print("="*70)

    for platform_name, post in posts.items():
        print(f"\n{'='*70}")
        print(f"📱 {platform_name.upper()} POST")
        print(f"{'='*70}")
        print(f"\n📊 Quality Scores:")
        print(f"   Overall:          {post.quality_score:.1f}%")
        print(f"   Data Reliability: {post.data_reliability_score:.1f}%")
        print(f"   Engagement:       {post.audience_engagement_score:.1f}%")
        print(f"   Narrative Flow:   {post.narrative_flow_score:.1f}%")
        print(f"\n📝 Content:\n")
        print(post.content)
        print(f"\n{'='*70}")

    print("\n✅ SUCCESS! All posts generated perfectly!\n")

except TranscriptExtractionError as e:
    print(f"\n❌ TRANSCRIPT ERROR: {e}")
    print("\n💡 Solutions:")
    print("   1. Try a different video ID")
    print("   2. Use verified IDs: u4ZoJKF_VuA, Ks-_Mh1QhMc, arj7oStGLkU")
    print("   3. Make sure video has captions enabled")

except APIError as e:
    print(f"\n❌ API ERROR: {e}")
    print("\n💡 Solutions:")
    print("   1. Check your API key is correct")
    print("   2. Get key: https://aistudio.google.com/app/apikey")
    print("   3. Make sure key starts with 'AIzaSy'")

except Exception as e:
    print(f"\n❌ UNEXPECTED ERROR: {e}")
    print(f"\n{traceback.format_exc()}")


🚀 Initializing agent...


📝 Generating posts...


📊 RESULTS

📱 INSTAGRAM POST

📊 Quality Scores:
   Overall:          76.7%
   Data Reliability: 90.0%
   Engagement:       90.0%
   Narrative Flow:   50.0%

📝 Content:

🤯 Did you know your body language speaks volumes, even before you utter a single word? That handshake, that glance – they're all being analyzed! Social scientists have been digging into this for years, and the results are mind-blowing.

Turns out, how we carry ourselves *seriously* impacts how people judge us, and these judgments can predict some pretty big stuff. Think hiring decisions, negotiation success, even how well your doctor gets along with patients! 🩺

Here's the kicker: it's not just about how others see you. Our nonverbal cues actually affect *our own* thoughts, feelings, and even our physiology! Power posing, anyone? 💪

So, what's the takeaway? Understanding and leveraging your nonverbal communication can be a game-changer in your career. Research shows that

## 💾 Step 10: Save Posts to Files (Optional)

In [10]:
# Save posts to text files
# Works with either 'posts' from Step 9 or 'my_posts' from Step 11
try:
    # Try to find posts from either test cell
    if 'my_posts' in locals():
        posts_to_save = my_posts
    elif 'posts' in locals():
        posts_to_save = posts
    else:
        print("⚠️ No posts to save yet!")
        print("💡 Run Step 9 or Step 11 first to generate posts")
        posts_to_save = None

    if posts_to_save:
        for platform_name, post in posts_to_save.items():
            filename = f"{platform_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"

            with open(filename, 'w', encoding='utf-8') as f:
                f.write(f"Platform: {platform_name}\n")
                f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Quality Score: {post.quality_score:.1f}%\n")
                f.write(f"\n{'-' * 70}\n\n")
                f.write(post.content)
                f.write(f"\n\n{'-' * 70}\n")
                f.write(f"\nData Sources: {len(post.story_structure.data_sources)}\n")

            print(f"✅ Saved {platform_name} post to: {filename}")

            # Download files (works in Colab)
            try:
                from google.colab import files
                files.download(filename)
            except:
                pass

        print("\n✅ All posts saved successfully!")

except Exception as e:
    print(f"❌ Error saving files: {str(e)}")

✅ Saved instagram post to: instagram_20251029_230914.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Saved linkedin post to: linkedin_20251029_230914.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ All posts saved successfully!


## 🎨 Step 11: Customize for Your Own Videos

In [11]:
# Use this cell to generate posts for YOUR videos

# Replace with your video URL or ID
MY_VIDEO = "YOUR_VIDEO_ID_OR_URL_HERE"

# Choose platforms
MY_PLATFORMS = [Platform.INSTAGRAM, Platform.LINKEDIN]
# Or just one: MY_PLATFORMS = [Platform.INSTAGRAM]

# Choose languages (tries in order)
MY_LANGUAGES = ['en']  # English
# Or: MY_LANGUAGES = ['en', 'es']  # Try English, then Spanish

# Generate posts
if MY_VIDEO != "YOUR_VIDEO_ID_OR_URL_HERE":
    try:
        print(f"\n🎬 Processing: {MY_VIDEO}\n")

        my_posts = agent.create_posts_from_video(
            video_url_or_id=MY_VIDEO,
            platforms=MY_PLATFORMS,
            languages=MY_LANGUAGES
        )

        # Show results
        for platform_name, post in my_posts.items():
            print(f"\n{'='*70}")
            print(f"📱 {platform_name.upper()}")
            print(f"{'='*70}")
            print(f"Quality: {post.quality_score:.1f}%\n")
            print(post.content)
            print(f"\n{'='*70}")

        print("\n✅ Your custom posts are ready!")

    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        print("\n💡 Tips:")
        print("   • Make sure video has transcripts")
        print("   • Try a verified video ID first")
        print("   • Check error message above")
else:
    print("💡 Replace MY_VIDEO with your video ID or URL to generate posts!")

💡 Replace MY_VIDEO with your video ID or URL to generate posts!


## 📚 Verified Working Videos

Copy these video IDs to test (confirmed working):

In [12]:
VERIFIED_VIDEOS = {
    "TED Talks": {
        "u4ZoJKF_VuA": "Simon Sinek - How great leaders inspire action",
        "Ks-_Mh1QhMc": "Amy Cuddy - Your body language shapes who you are",
        "arj7oStGLkU": "Tim Urban - Inside the mind of a procrastinator",
    },
    "Educational": {
        "aircAruvnKk": "3Blue1Brown - What is a neural network?",
        "GiPe1OiKQuk": "Crash Course - Intro to Machine Learning",
    }
}

print("📺 VERIFIED WORKING VIDEO IDs:\n")
for category, videos in VERIFIED_VIDEOS.items():
    print(f"\n{category}:")
    for video_id, description in videos.items():
        print(f"  • {video_id}")
        print(f"    {description}")
        print(f"    https://youtube.com/watch?v={video_id}")

📺 VERIFIED WORKING VIDEO IDs:


TED Talks:
  • u4ZoJKF_VuA
    Simon Sinek - How great leaders inspire action
    https://youtube.com/watch?v=u4ZoJKF_VuA
  • Ks-_Mh1QhMc
    Amy Cuddy - Your body language shapes who you are
    https://youtube.com/watch?v=Ks-_Mh1QhMc
  • arj7oStGLkU
    Tim Urban - Inside the mind of a procrastinator
    https://youtube.com/watch?v=arj7oStGLkU

Educational:
  • aircAruvnKk
    3Blue1Brown - What is a neural network?
    https://youtube.com/watch?v=aircAruvnKk
  • GiPe1OiKQuk
    Crash Course - Intro to Machine Learning
    https://youtube.com/watch?v=GiPe1OiKQuk


## 🎉 You're Done!

### Next Steps:
1. **Customize the tone** - Edit the prompts in Step 6
2. **Use your videos** - Add your video IDs in Step 11
3. **Automate** - Schedule this to run weekly
4. **Scale up** - Process multiple videos at once
5. **Make it billingual** - Accept both Spanish and English inputs and outputs, or translate Spanish to English for post generation
6. **Iterations** - Add more layers of self-criticism and higher benchmarks of quality
### Tips:
- Posts with 70%+ quality are ready to publish
- Always review before posting
- Add your personal touch to the AI-generated content
- Test different videos to see what works best

### Need Help?
- **API Key:** https://aistudio.google.com/app/apikey
- **Docs:** https://googleapis.github.io/python-genai/
- **YouTube API:** https://github.com/jdepoix/youtube-transcript-api

---

**🎊 Happy Automating!**

*This notebook uses the FIXED version that works with youtube-transcript-api v1.2.0+*